In [1]:
# Obtain parameters in Tidal Cases
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# Melting flux at ice front
def IFA(case_path, case_id):
    # Grid areas    
    Area = np.empty([90, 10])
    Area[:20,:] = 400
    Area[20:50,:] = 800
    Area[50:,:] = 1200

    file0 = xr.open_dataset(case_path + '/icefrntA_' + str(format(case_id,'03d')) + '.nc')
    t0 = 89
    #tn = 120
    # removed duplicated data caused by model restart
    file = file0.isel(T=~file0.get_index("T").duplicated())
    state = file.isel(Y=range(35,45), T=range(t0,len(file.T)))
    
    MR = state.icefrntA.isel(X=1).data.mean(0) # Melt rate at the icefront
    Qsm = (MR*Area).sum()/(24*3600)
    return Qsm

In [ ]:
# Water temperature at ice front
def icefrntT(case_path, case_id):
    File1 = xr.open_dataset(case_path + '/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(case_path + '/grid_' + str(format(case_id,'03d')) + '.nc')  
    State = File1.isel(T=~File1.get_index("T").duplicated())
    
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))

    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2 # Along-channel velocity
    
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
   
    t0 = 89 # Starting time index
    da = DA[t0:,:,:,1].mean(axis=(0,2))
    Ti = state.Temp.data[t0:,:,:,1].mean(axis=(0,2))
    Ui = U[t0:,:,:,1].mean(axis=(0,2))
    
    Tvi = np.sum(Ti*da) / np.sum(da)
    return Tvi

In [ ]:
path1 = '/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs'
path2 = '/work/oceans/wbao/MITgcm_results/iceplume/weak_Tide_minhs' # Qsg=250


caseN1 = np.array([1,2,3,4,5])
Qsm1 = np.empty(len(caseN1))
Tice1 = np.empty(len(caseN1))

caseN2 = np.array([1,2,3])
Qsm2 = np.empty(len(caseN2))
Tice2 = np.empty(len(caseN2))

for i in range(len(caseN1)):
    
    Qsm1[i] = IFA(path1, caseN1[i])
    Tice1[i] = icefrntT(path1, caseN1[i])

    
for j in range(len(caseN2)):
    
    Qsm2[j] = IFA(path2, caseN2[j])
    Tice2[j] = icefrntT(path2, caseN2[j])

In [ ]:
Qsm2, Qsm1, Tice2, Tice1

In [2]:
file1 = xr.open_dataset('/home/1959/Parameters/FjordModeling/Tide01.nc')
file2 = xr.open_dataset('/home/1959/Parameters/FjordModeling/Tide02.nc')
file3 = xr.open_dataset('/home/1959/Parameters/FjordModeling/Tide03.nc')

a11 = np.concatenate([file1.alpha11.data, file2.alpha11.data, file3.alpha11.data])
a22 = np.concatenate([file1.alpha22.data, file2.alpha22.data, file3.alpha22.data])

Qin1 = np.concatenate([file1.Qin1.data, file2.Qin1.data, file3.Qin1.data])
Qin1_adj = np.concatenate([file1.Qin1_adj.data, file2.Qin1_adj.data, file3.Qin1_adj.data])

Qin2 = np.concatenate([file1.Qin2.data, file2.Qin2.data, file3.Qin2.data])
Qin2_adj = np.concatenate([file1.Qin2_adj.data, file2.Qin2_adj.data, file3.Qin2_adj.data])

Qout1 = np.concatenate([file1.Qout1.data, file2.Qout1.data, file3.Qout1.data])
Qout1_adj = np.concatenate([file1.Qout1_adj.data, file2.Qout1_adj.data, file3.Qout1_adj.data])

Qout2 = np.concatenate([file1.Qout2.data, file2.Qout2.data, file3.Qout2.data])
Qout2_adj = np.concatenate([file1.Qout2_adj.data, file2.Qout2_adj.data, file3.Qout2_adj.data])

Ts_in = np.concatenate([file1.Ts_in.data, file2.Ts_in.data, file3.Ts_in.data])
Ts = np.concatenate([file1.Ts.data, file2.Ts.data, file3.Ts.data])
Tf = np.concatenate([file1.Tf.data, file2.Tf.data, file3.Tf.data])

Qsm = np.concatenate([file1.Qsm.data, file2.Qsm.data, file3.Qsm.data])

In [37]:
a11, a22, (Qin1_adj-Qout2_adj)/Qin1_adj, Qin1_adj-Qout2_adj, Qin1_adj, Qout2_adj
#(Qin2+Qout2)/2, Qin1, Qout1

(array([0.714578  , 0.71510556, 0.71483592, 0.72686785, 0.74121893,
        0.74971925, 0.76033412, 0.76944392, 0.80404705,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan]),
 array([0.04379473, 0.04338967, 0.04361214, 0.16954694, 0.27852152,
        0.36469475, 0.42617604, 0.48368894, 0.67170154,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan]),
 array([0.70277548, 0.70344345, 0.70310735, 0.67694333, 0.65246778,
        0.62301338, 0.60430422, 0.58125379, 0.46698783, 0.33123252,
        0.17610844, 0.13322018, 0.21017625, 0.23734492, 0.28466557,
        0.31458973, 0.23911683]),
 array([ 6329.56148222,  6347.55159965,  6339.53758782,  6104.48178636,
         5966.36378861,  5719.40729499,  5712.3838591 ,  5594.89984592,
         4851.71897906,  4099.58097071,  2469.06566866,  2526.82063983,
         5228.62158002,  6510.88512667,  9126.05378112, 11154.19859146

In [ ]:
file1 = xr.open_dataset('/home/1959/Parameters/FjordModeling/4_wTide_minhs.nc')
file2 = xr.open_dataset('/home/1959/Parameters/FjordModeling/4_sTide_minhs.nc')

n = -1
a11 = np.concatenate([file1.alpha11.data, file2.alpha11.data[:]])
a22 = np.concatenate([file1.alpha22.data, file2.alpha22.data[:]])

Qin1 = np.concatenate([file1.Qin1.data, file2.Qin1.data[:]])
Qin1_adj = np.concatenate([file1.Qin1_adj.data, file2.Qin1_adj.data[:]])

Qin2 = np.concatenate([file1.Qin2.data, file2.Qin2.data[:]])
Qin2_adj = np.concatenate([file1.Qin2_adj.data, file2.Qin2_adj.data[:]])

Qout1 = np.concatenate([file1.Qout1.data, file2.Qout1.data[:]])
Qout1_adj = np.concatenate([file1.Qout1_adj.data, file2.Qout1_adj.data[:]])

Qout2 = np.concatenate([file1.Qout2.data, file2.Qout2.data[:]])
Qout2_adj = np.concatenate([file1.Qout2_adj.data, file2.Qout2_adj.data[:]])

Ts_in = np.concatenate([file1.Ts_in.data, file2.Ts_in.data[:]])
Ts = np.concatenate([file1.Ts.data, file2.Ts.data[:]])
Tf = np.concatenate([file1.Tf.data, file2.Tf.data[:]])

Qsm = np.concatenate([file1.Qsm.data, file2.Qsm.data[:]])

In [ ]:
file2

In [ ]:
#(Qin1_adj+Qout1_adj)/2, (Qin2+Qout2)/2, (Qin1-Qout2), (Qin1-Qout2)/Qin1*100
a11, a22, (Qin1_adj-Qout2_adj)/Qin1_adj, Ts_in, Ts

In [ ]:
parameter_ds = xr.Dataset(
    data_vars={'U0' : np.array([1e-4, 2e-4, 3e-4, 1e-3, 2e-3]),
    'alpha11' : (Qin1_adj-Qout2_adj)/Qin1_adj,
    'alpha22' : a22,
    'Qin1' : Qin1_adj,
    'Qin2' : Qin2_adj,
    'Qout1' : Qout1_adj,
    'Qout2' : Qout2_adj,
    'Ts_in' : Ts_in,
    'Tf' : Tf,
    'Ts_model' : Ts})

outdir = "/home/1959/Parameters/FjordModeling/Eq4/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

parameter_ds.to_netcdf(outdir + '4-Tide.nc')

In [ ]:
Ts/Ts_in*100, Qsm

In [25]:
file3 = xr.open_dataset('/home/1959/Parameters/FjordModeling/1_BaseCase.nc')
a11b = file3.alpha11.data
a22b = file3.alpha22.data
Qin1b = file3.Qin1.data
Qin2b = file3.Qin2.data
Qout1b = file3.Qout1.data
Qout2b = file3.Qout2.data
Qsmb = file3.Qsm.data
Tsb = file3.Ts.data
Ts_inb = file3.Ts_in.data

In [36]:
a11b, a22b, (Qin2b+Qout2b)/2, file3.Qin1_adj.data, file3.Qout2_adj.data

(array([0.71530895, 0.56250147, 0.42047409, 0.29042472, 0.17951959,
        0.01136236]),
 array([ 0.04242288,  0.00787062, -0.01761308, -0.04125368, -0.02628035,
         0.00991505]),
 array([2548.01896578, 3872.47116847, 4963.0527807 , 5844.6496557 ,
        6406.33548218, 7160.25633201]),
 array([9024.81065508, 9069.39780591, 8925.31153739, 8745.61446087,
        8160.76748157, 7358.33371601]),
 array([2672.11268784, 3997.34417539, 5087.22163861, 5969.71802319,
        6530.65636717, 7343.89914516]))

In [ ]:
case_path = '/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs'
case_id = 5
State0 = xr.open_dataset(case_path + '/state_' + str(format(case_id,'03d')) + '.nc')
State = State0.isel(T=~State0.get_index("T").duplicated())
state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
eta = state.Eta.data

In [ ]:
eta.max(), eta.min(), eta.mean()

In [19]:
import gsw
def ambient_strat(datapath, case_id):
    
    State01 = xr.open_dataset(datapath + '/state_' + str(format(case_id,'03d')) + '.nc')    
    State = State01.isel(T=~State01.get_index("T").duplicated())
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), Yp1=range(35,45), T=range(89,len(State.T)))
    
    xidx = 2 # Glacier front index
    
    # Potential temp
    T0 = state.Temp.data[:,:,:,xidx].mean(axis=(0,2))
    
    # Salinity
    S0 = state.S.data[:,:,:,xidx].mean(axis=(0,2))
    
    depth = state.Z.data  
    # Pressure
    pres = gsw.p_from_z(depth, 55)

    N2, Pmid = gsw.Nsquared(S0,T0,pres,55)
    
    return N2.mean()

In [22]:
path1 = '/work/oceans/wbao/MITgcm_results/iceplume/weak_Tide_minhs'
path2 = '/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs'
path3 = '/work/oceans/wbao/MITgcm_results/iceplume/6_sTide_minhs'
case1 = np.arange(1,4)
case2 = np.arange(1,6)
case3 = np.array([1,2,3,5,6,7,8,9,10])
strat1 = np.empty(len(case1))
strat2 = np.empty(len(case2))
strat3 = np.empty(len(case3))

for i in range(len(case1)):
    strat1[i] = ambient_strat(path1, case1[i])

for i in range(len(case2)):
    strat2[i] = ambient_strat(path2, case2[i])

for i in range(len(case3)):
    strat3[i] = ambient_strat(path3, case3[i])

tide_strat = np.concatenate([strat1, strat2, strat3])

In [23]:
path4 = '/work/oceans/wbao/MITgcm_results/iceplume/1_BaseCase'
tide0_strat = ambient_strat(path4, 1)

In [24]:
tide_strat, tide0_strat

(array([6.99697488e-05, 6.99685338e-05, 6.99712474e-05, 6.99737120e-05,
        6.99724415e-05, 6.99992843e-05, 7.00097469e-05, 7.00284034e-05,
        6.94419161e-05, 6.69205224e-05, 6.30316177e-05, 5.81249774e-05,
        5.54740035e-05, 5.14967310e-05, 4.82219234e-05, 4.56854056e-05,
        4.44674893e-05]),
 6.997267818520045e-05)